In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import random
import pandas as pd
import re
from dateutil import parser as date_parser

In [15]:
# Configure Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run headless (optional)
chrome_options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
    "AppleWebKit/537.36 (KHTML, like Gecko) "
    "Chrome/90.0.4430.93 Safari/537.36"
)
# Set browser language to English
chrome_options.add_experimental_option("prefs", {"intl.accept_languages": "en-US"})

# Set the path to your chromedriver
service = Service(executable_path=r"C:\Users\anuar\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe")
driver = webdriver.Chrome(service=service, options=chrome_options)
results = []
try:
    url = "https://www.coursera.org/search?query=&"
    driver.get(url)
    
    # Reduce zoom level to see more results on the screen
    driver.execute_script("document.body.style.zoom='1%'")
    
    start_time = time.time()

    while time.time() - start_time < 5000:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # wait for new content to load
    
    
    # Find all product cards
    product_cards = driver.find_elements(By.CSS_SELECTOR, "div.cds-ProductCard-gridCard")
    print(f"Found product cards: {len(product_cards)}")
    
    # Iterate over each product card and extract the URL from the <a> tag and the text of the card
    for card in product_cards[:]:
        try:
            a_tag = card.find_element(By.TAG_NAME, "a")
            card_url = a_tag.get_attribute("href")
        except Exception as e:
            card_url = None
        card_text = card.text.split('\n')
        try:
            record = {
            "url": card_url,
            "course_name": card_text[-7] if len(card_text) > 8 and card_text[-9] != 'New' else (card_text[-7] if card_text[-5] == 'Build toward a degree' else card_text[-6] ) ,
            "rating": card_text[-4],
            "survey_count": card_text[-2],
            "Institution":  card_text[-8] if len(card_text) > 8 and card_text[-9] != 'New' else (card_text[-8] if card_text[-5] == 'Build toward a degree' else card_text[-7] ) ,
            "Subject": None,
            "Level": card_text[-1].split()[0],
            "Associated_skills": card_text[-6] if len(card_text) > 8 and card_text[-9] != 'New' else (card_text[-6] if card_text[-5] == 'Build toward a degree' else card_text[-5] ) ,
            "price": None,
            "duration": ' '.join(card_text[-1].split()[-4::])
            }
            results.append(record)
        except:
            try:
                record = {
                "url": card_url,
                "course_name": card_text[-3] if len(card_text) == 5 and card_text[0] == 'New' else (card_text[-3] if '-' not in card_text[-4] else card_text[-4].split('-')[1] ),
                "rating": None,
                "survey_count": None,
                "Institution": card_text[-4] if len(card_text) == 5 and card_text[0] == 'New' else  (card_text[-4] if '-' not in card_text[-4] else card_text[-4].split('-')[0] ) ,
                "Subject": None,
                "Level": card_text[-1].split()[0],
                "Associated_skills": card_text[-2] ,
                "price": None,
                "duration": ' '.join(card_text[-1].split()[-4::])
                }
                results.append(record)
            except:
                for i in range(len(card_text)):
                    print(len(card_text)-i, card_text[i])
                print("------------------------")
finally:
    driver.quit()

df = pd.DataFrame(results)
df.to_csv("coursera_df.csv", index=False)
df

,url,course_name,rating,survey_count,Institution,Subject,Level,Associated_skills,price,duration
0,https://www.coursera.org/professional-certific...,Google Data Analytics,4.8,163K reviews,Google,NaN,Beginner,"Skills you'll gain: Data Storytelling, Rmarkdo...",NaN,3 - 6 Months
1,https://www.coursera.org/professional-certific...,Google Cybersecurity,4.8,47K reviews,Google,NaN,Beginner,"Skills you'll gain: Threat Modeling, Open Web ...",NaN,3 - 6 Months
2,https://www.coursera.org/professional-certific...,Google Project Management:,4.8,124K reviews,Google,NaN,Beginner,"Skills you'll gain: Quality Management, Projec...",NaN,3 - 6 Months
3,https://www.coursera.org/professional-certific...,Google Digital Marketing & E-commerce,4.8,37K reviews,Google,NaN,Beginner,"Skills you'll gain: Media Planning, Search Eng...",NaN,3 - 6 Months
4,https://www.coursera.org/professional-certific...,Google UX Design,4.8,88K reviews,Google,NaN,Beginner,"Skills you'll gain: Storyboarding, Wireframing...",NaN,3 - 6 Months
...,...,...,...,...,...,...,...,...,...,...
7889,https://www.coursera.org/learn/windows-server-...,Windows Server Management and Security,4.7,654 reviews,University of Colorado System,NaN,Beginner,"Skills you'll gain: Windows Servers, Authoriza...",NaN,1 - 3 Months
7890,https://www.coursera.org/learn/math-for-democracy,"The Mathematics of Democracy, Politics and Man...",4.9,12 reviews,Johns Hopkins University,NaN,Mixed,"Skills you'll gain: Political Sciences, Policy...",NaN,1 - 3 Months
7891,https://www.coursera.org/learn/cuda-at-scale-f...,CUDA at Scale for the Enterprise,3.1,14 reviews,Johns Hopkins University,NaN,Intermediate,"Skills you'll gain: Image Analysis, Event-Driv...",NaN,1 - 3 Months
7892,https://www.coursera.org/learn/ingles-empresar...,Inglés Empresarial: Gestión y Liderazgo,4.8,1.7K reviews,Arizona State University,NaN,Mixed,"Skills you'll gain: Meeting Facilitation, Tele...",NaN,1 - 3 Months
